In [1]:
import os 
import sys 
import logging 
from functools import cache 

logging.basicConfig(level=logging.INFO)

cur_path = os.path.abspath("../..")
if cur_path not in sys.path: 
    sys.path.append(cur_path)

import numpy as np 
import pandas as pd 
import altair as alt 
from altair import datum
from dotenv import load_dotenv
from subgrounds.subgrounds import Subgrounds, Subgraph
from subgrounds.subgraph import SyntheticField
from subgrounds.pagination import ShallowStrategy
from python_graphql_client import GraphqlClient

# Required when developing in a jupyter-notebook environment 
assert load_dotenv('../../../.env')

from utils_notebook.utils import remove_prefix, ddf, load_subgraph
from utils_notebook.vega import output_chart
from utils_notebook.queries import QueryManager
from utils_notebook.testing import validate_season_series

In [2]:
sg, bs = load_subgraph()
q = QueryManager(sg, bs)

In [3]:
# TODO: Figure out what the hell this is and whether or not we should keep it. 

# df_ftokens = df_fert_tokens_raw.copy()

# nearest = alt.selection_single(
#     nearest=True, encodings=["x"], on="mouseover", empty="none"
# )
# brush = alt.selection_interval(encodings=['x'])

# base = alt.Chart(df_ftokens)
# x = alt.X("humidity:Q")
# y = alt.Y("fertilizer:Q")

# # Histogram of fertilizer purchased by season 
# histogram = base.mark_bar().encode(
#     x=x, y=y
# )

# # Vertical line to indicate nearest histogram bin 
# vline = base.mark_rule(color="grey").encode(
#     x=x, size=alt.value(1)
# ).transform_filter(
#     nearest
# )

# # Text label to indicate value nearest histogram bin 
# text = histogram.mark_text(align='left', dx=5, dy=-5).encode(
#     text=alt.condition(nearest, y, alt.value(' '))
# )

# # Time axis which drives scale of histogram 
# time_axis = base.mark_bar().encode(
#     x=x, y=y
# ).add_selection(
#     brush 
# ).resolve_scale(
#     'independent'
# ).properties(height=50)

# (
#     alt.layer(
#         histogram.add_selection(nearest), 
#         text, 
#         vline
#     ).transform_filter(
#         brush
#     ).properties(height=250) 
#     & time_axis 
# ) 

In [4]:
df = q.query_barn()

INFO:subgrounds:client.query: url = https://api.thegraph.com/subgraphs/name/cujowolf/beanstalk, variables = {'first0': 900, 'skip0': 0}
query($first0: Int, $skip0: Int) {
  xed1126c9f5e7040a: seasons(first: $first0, skip: $skip0, orderBy: season, orderDirection: asc, where: {season_gte: 6074}) {
    season
    timestamp
    id
  }
}
INFO:subgrounds:client.query: url = https://api.thegraph.com/subgraphs/name/cujowolf/beanstalk, variables = {'first0': 900, 'skip0': 0, 'lastOrderingValue0': 6973}
query($first0: Int, $skip0: Int, $lastOrderingValue0: Int) {
  xed1126c9f5e7040a: seasons(first: $first0, skip: $skip0, orderBy: season, orderDirection: asc, where: {season_gte: 6074, season_gt: $lastOrderingValue0}) {
    season
    timestamp
    id
  }
}
INFO:subgrounds:client.query: url = https://api.thegraph.com/subgraphs/name/cujowolf/beanstalk, variables = {'first0': 900, 'skip0': 0}
query($first0: Int, $skip0: Int) {
  x9e2abc3720c06e78: rewards(first: $first0, skip: $skip0, orderBy: block

In [11]:
def visualize_recap_paid_percent(df: pd.DataFrame): 
    """"""
    df = df.iloc[1:,]
    # fertilizer breakdown 
    series_fert = df.iloc[-1,][['fertilizer_available', 'fertilizer_active']]
    fert_total = series_fert['fertilizer_available'] + series_fert['fertilizer_active']
    df_fert = pd.DataFrame(
        {
            'category': series_fert.index, 'value': series_fert.values, 
            'label': [
                f"{c}: {int(v):,} \n({v / fert_total:.1%})" 
                for c, v in zip(series_fert.index, series_fert.values)
            ]
        }
    )
    fertililzer_breakdown = alt.Chart(
        df_fert, title="Fertilizer Breakdown"
    ).encode(
        theta=alt.Theta("value:Q", stack=True),
        color=alt.Color("category:N", legend=alt.Legend()),
        tooltip="label:N"
    ).mark_arc(outerRadius=120, innerRadius=75)
    # sprouts breakdown 
    df_sprouts = df.copy()
    label_vars = ['sprouts rinsable', 'sprouts ', 'rinsable percent']
    id_vars = ['timestamp'] + label_vars
    value_vars = ['sprouts_rinsable', 'sprouts', 'rinsable_percent']
    df_sprouts['sprouts rinsable'] = df_sprouts['sprouts_rinsable']
    df_sprouts['sprouts '] = df_sprouts['sprouts']
    df_sprouts['rinsable percent'] = df_sprouts['rinsable_percent']
    df_sprouts = df_sprouts.melt(
        id_vars=id_vars, value_vars=value_vars
    ).sort_values("timestamp")
    sprouts_base = alt.Chart(
        df_sprouts[['timestamp', 'value', 'variable'] + label_vars], 
        title="Sprouts Breakdown"
    ).encode(
        x=alt.X("yearmonthdatehours(timestamp):T", axis=alt.Axis(tickCount=10, title="Date")), 
        y=alt.Y("value:Q", axis=alt.Axis(title="Sprouts")), 
        color=alt.Color("variable:N", legend=alt.Legend()),
    )
    sprouts_area = sprouts_base.transform_filter(
        datum.variable != 'rinsable_percent'
    ).mark_area(point="transparent").encode(
        tooltip=[
            alt.Tooltip(f'{e}:Q', format=",d" if e != 'rinsable percent' else ".2%") 
            for e in label_vars
        ]
    )
    sprouts_line = sprouts_base.transform_filter(
        datum.variable == 'rinsable_percent'
    ).mark_line().encode(
        y=alt.Y("value:Q", axis=alt.Axis(title="Rinsable Percent", format=".2%")), 
    )
    sprouts_breakdown = alt.layer(sprouts_area, sprouts_line).resolve_scale(
        y="independent", 
    ).resolve_axis(
        y="independent"
    ).properties(width=300)
    
    c = alt.hconcat(
        sprouts_breakdown, 
        fertililzer_breakdown
    ).resolve_legend(
        color="independent", 
    ).resolve_scale(
        y="independent",
        color="independent",
    ).resolve_axis(
        y="independent"
    )
    return c

c = visualize_recap_paid_percent(df)
c

alt.HConcatChart(...)

In [12]:
output_chart(c)

<IPython.core.display.JSON object>